In [32]:
from decoders.interleaved import read_pcap
from utils.matlab import db
from tensorflow.keras import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
import glob
from numpy import inf
from tensorflow.keras.layers import Bidirectional


import numpy as np
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
import os

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold

from scipy.interpolate import interp1d

In [2]:
base_dir = "."
dataset_dir = os.path.join("data")

In [58]:
FS = 100
ALL_ACTIVITIES = ["static", "standing", "walking"]

In [158]:
def loadFromDat(inputFile, windowSize = FS, step = 50):
#     print(int((csi_data.nsamples-1)*(100/average_sample_rate)+1))
    
    csi_data = read_pcap(inputFile)
    csi = csi_data.csi
    
    
    first_timestamp = float(csi_data.timestamps[0])
    last_timestamp = float(csi_data.timestamps[-1])
    final_timestamp = last_timestamp-first_timestamp
    average_sample_rate = csi_data.nsamples/final_timestamp
    
    interp_func=interp1d(csi_data.timestamps, csi_data.csi, kind='linear', axis=0, fill_value="extrapolate")
    t_new = np.linspace(first_timestamp, last_timestamp, (csi_data.nsamples-1)*int((100/average_sample_rate)+1))
    csi_interp = interp_func(t_new)
    csi=csi_interp
    
    csi = db(np.abs(csi))
    finalData = csi[:, :, 0]
    finalData = np.transpose(finalData)
    
    new_average_sample_rate = len(csi_interp)/final_timestamp
    
    if new_average_sample_rate > FS:
        downsample_factor = int(new_average_sample_rate/FS)
        csi = csi[::downsample_factor]
    index = 0
    positiveInput = []
    
    while index + windowSize <= csi.shape[0]:
        curFeature = np.zeros((1, windowSize, 256))
        curFeature[0] = csi[index:index+windowSize, :].reshape(100,256)
        positiveInput.append(curFeature)
        index += step
    try:
        positiveInput = np.concatenate(positiveInput, axis = 0)
    except ValueError as e:
        positiveInput = np.zeros((1, windowSize, 256))
    positiveInput[positiveInput == -inf] = 0
    
    return positiveInput

In [52]:
def loadByActivityLabel(directory, activity, activities=ALL_ACTIVITIES, windowSize=FS, step = 50, downsample=1):
    activity = activity.lower()
    if activity not in activities:
        print("invalid activity: " + activity)
    
    dataPathPattern = os.path.join(directory,"{}*.pcap".format(activity))
    inputFiles = sorted(glob.glob(dataPathPattern, recursive =True))
    
    inputWindows = []
    index = 0
    for inputFile in inputFiles:
        index += 1
        csiOutput = loadFromDat(inputFile, windowSize = windowSize, step=step)
        if csiOutput is not None:
            inputWindows.append(csiOutput)
    inputArray = np.concatenate(inputWindows, axis = 0)
    outputLabels = np.zeros((inputArray.shape[0], len(activities)))
    outputLabels[:, activities.index(activity)] = 1
    return inputArray, outputLabels


In [53]:
def loadDataForActivities(directory, activities=ALL_ACTIVITIES, windowSize=100, step=50, downsample=1):
    x_all = []
    y_all = []
    for activity in activities:
        inputArray, outputLabels = loadByActivityLabel(directory, activity, activities, windowSize=windowSize, step=step, downsample=downsample)
        x_all.append(inputArray)
        y_all.append(outputLabels)
    x_all = np.concatenate(x_all, axis=0)
    y_all = np.concatenate(y_all, axis=0)
    return (x_all, y_all)

In [159]:
x_all, y_all = loadDataForActivities(dataset_dir, downsample=1)


In [160]:
loadDataForActivities(dataset_dir, downsample=1)

(array([[[75.82117487, 91.1968139 , 42.14419939, ..., 24.4870632 ,
          23.97940009, 17.63427994],
         [75.82117487, 91.1968139 , 42.14419939, ..., 25.13374666,
          18.82639991, 23.98065317],
         [75.82117487, 91.1968139 , 42.14419939, ..., 25.95279419,
          15.27814445, 23.73498563],
         ...,
         [75.82117487, 91.18616539, 42.14419939, ..., 24.09829638,
          20.2568525 , 20.38573453],
         [75.82117487, 91.19448347, 42.14419939, ..., 27.89504509,
          29.00928825, 22.21924811],
         [75.82117487, 91.1968139 , 42.14419939, ..., 27.52979228,
          27.65363008, 10.51237856]],
 
        [[75.82117487, 91.1968139 , 42.14419939, ..., 14.76296322,
          15.05162086, 15.90466465],
         [75.82117487, 91.1968139 , 42.14419939, ..., 21.79406852,
          12.90583814,  6.44155676],
         [75.82117487, 91.1968139 , 42.14419939, ..., 22.47093636,
          -1.78505337, 10.7315378 ],
         ...,
         [75.82117487, 91.1968139

In [161]:
x_train, x_val, y_train, y_val = train_test_split(x_all, y_all, stratify=y_all, test_size = 0.3, random_state=42)

In [162]:
batch_size = 256
epochs = 100

f1s = []
confs = []
y_val_total = np.array([])
y_pred_total = np.array([])


In [163]:
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=5, activation='relu', strides=1))
model.add(MaxPooling1D(pool_size=4))
# model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
# model.add(MaxPooling1D(pool_size=4))
model.add(Bidirectional(LSTM(units=200, return_sequences=False)))
# model.add(Bidirectional(LSTM(units=200, return_sequences=False)))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))


In [169]:
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

In [165]:
model.fit(x_train, y_train, batch_size=256, epochs=100, shuffle=False, validation_data=(x_val, y_val))

Epoch 1/100
4/4 [==============================] - 11s 971ms/step - loss: 1.0456 - accuracy: 0.4840 - val_loss: 0.9089 - val_accuracy: 0.5300
Epoch 2/100
4/4 [==============================] - 2s 584ms/step - loss: 0.9346 - accuracy: 0.4923 - val_loss: 0.9393 - val_accuracy: 0.5444
Epoch 3/100
4/4 [==============================] - 2s 590ms/step - loss: 0.9068 - accuracy: 0.5953 - val_loss: 0.7994 - val_accuracy: 0.7482
Epoch 4/100
4/4 [==============================] - 2s 561ms/step - loss: 0.8632 - accuracy: 0.5757 - val_loss: 0.7887 - val_accuracy: 0.8369
Epoch 5/100
4/4 [==============================] - 2s 547ms/step - loss: 0.7675 - accuracy: 0.7549 - val_loss: 0.7937 - val_accuracy: 0.6954
Epoch 6/100
4/4 [==============================] - 2s 540ms/step - loss: 0.7393 - accuracy: 0.6993 - val_loss: 0.7224 - val_accuracy: 0.7938
Epoch 7/100
4/4 [==============================] - 2s 533ms/step - loss: 0.6974 - accuracy: 0.7456 - val_loss: 0.8112 - val_accuracy: 0.6859
Epoch 8/100


Epoch 59/100
4/4 [==============================] - 2s 573ms/step - loss: 0.3699 - accuracy: 0.8888 - val_loss: 0.3913 - val_accuracy: 0.9281
Epoch 60/100
4/4 [==============================] - 2s 561ms/step - loss: 0.4738 - accuracy: 0.7600 - val_loss: 0.4919 - val_accuracy: 0.8321
Epoch 61/100
4/4 [==============================] - 2s 539ms/step - loss: 0.4871 - accuracy: 0.7971 - val_loss: 0.4538 - val_accuracy: 0.7722
Epoch 62/100
4/4 [==============================] - 2s 547ms/step - loss: 0.4793 - accuracy: 0.8290 - val_loss: 0.3458 - val_accuracy: 0.9400
Epoch 63/100
4/4 [==============================] - 2s 518ms/step - loss: 0.4703 - accuracy: 0.7611 - val_loss: 0.3904 - val_accuracy: 0.8585
Epoch 64/100
4/4 [==============================] - 2s 548ms/step - loss: 0.3683 - accuracy: 0.8641 - val_loss: 0.3495 - val_accuracy: 0.9496
Epoch 65/100
4/4 [==============================] - 2s 528ms/step - loss: 0.3261 - accuracy: 0.9125 - val_loss: 0.3758 - val_accuracy: 0.8657
Epoch 

In [166]:
y_pred = model.predict(x_val)

14/14 [==============================] - 1s 29ms/step


In [167]:
y_val_single = np.argmax(y_val, axis=1)
y_pred_single = np.argmax(y_pred, axis=1)

In [168]:
print(confusion_matrix(y_val_single, y_pred_single))
print(classification_report(y_val_single, y_pred_single, target_names=["static", "standing", "walking"]))

[[164   0   1]
 [  0  86   0]
 [  0  27 139]]
              precision    recall  f1-score   support

      static       1.00      0.99      1.00       165
    standing       0.76      1.00      0.86        86
     walking       0.99      0.84      0.91       166

    accuracy                           0.93       417
   macro avg       0.92      0.94      0.92       417
weighted avg       0.95      0.93      0.93       417

